In [2]:
# Introdução
O crescimento dos sistemas de comércio eletrônico trouxe desafios para os consumidores diante da grande variedade de produtos e serviços disponíveis. Os Sistemas de Recomendação (SR) auxiliam os usuários a encontrar itens relevantes, sendo foco de diversas pesquisas, especialmente na melhoria da acurácia dos algoritmos. Este trabalho visa implementar e comparar diferentes algoritmos de filtragem colaborativa, abordando também questões de hiperparametrização e avaliação de desempenho.

SyntaxError: invalid syntax (3476877927.py, line 2)

# Experimentos

## Bases de Dados
- Utilizaremos a base MovieLens 100k para treinamento e avaliação dos modelos.

## Metodologia
- Implementação de algoritmos de filtragem colaborativa baseados em memória (user-based) e baseados em modelo (Regularized SVD).
- Ajuste de hiperparâmetros para otimização do desempenho.
- Avaliação utilizando métricas apropriadas (ex: RMSE, MAE).

## Algoritmos Implementados
- Filtragem colaborativa baseada em memória (User-based).
- Filtragem colaborativa baseada em modelo (SVD regularizado).

# Conclusão

Serão apresentadas as principais conclusões, análise dos resultados e sugestões para trabalhos futuros.

In [5]:
# Instalação e importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Outras bibliotecas serão importadas conforme necessário

In [6]:
# Download e carregamento da base MovieLens 100k
import zipfile
import os
import urllib.request

url = 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
filename = 'ml-100k.zip'

if not os.path.exists('ml-100k'):
    urllib.request.urlretrieve(url, filename)
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('.')

ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
# Filtragem colaborativa baseada em memória (User-based)
from sklearn.neighbors import NearestNeighbors

# Criar matriz usuário-item
user_item_matrix = ratings.pivot_table(index='user_id', columns='item_id', values='rating').fillna(0)

# Treinar modelo KNN
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(user_item_matrix)

# Exemplo: encontrar os 5 usuários mais similares ao usuário 1
user_index = 0  # índice do usuário 1
n_neighbors = 5
neighbors = knn.kneighbors([user_item_matrix.iloc[user_index]], n_neighbors=n_neighbors+1)
print(f'Usuários mais similares ao usuário 1: {neighbors[1][0][1:]}')

Usuários mais similares ao usuário 1: [915 863 267  91 434]


In [8]:
# Filtragem colaborativa baseada em modelo (SVD regularizado)
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, GridSearchCV

# Preparar dados para Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

# Treinar modelo SVD
svd = SVD()
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
print('Resultados SVD:', results)

ModuleNotFoundError: No module named 'surprise'

In [ ]:
# Ajuste de hiperparâmetros do SVD
param_grid = {'n_factors': [50, 100], 'reg_all': [0.02, 0.1]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print('Melhores hiperparâmetros:', gs.best_params['rmse'])
print('Melhor RMSE:', gs.best_score['rmse'])

In [4]:
# Avaliação dos modelos e visualização dos resultados
import matplotlib.pyplot as plt

# Exemplo de comparação de RMSE e MAE dos modelos
svd_rmse = gs.best_score['rmse']
svd_mae = gs.best_score['mae']

# Para o modelo user-based, pode-se calcular RMSE/MAE manualmente após gerar previsões
# Aqui, apenas um exemplo ilustrativo
user_based_rmse = 1.0  # Substitua pelo valor real após avaliação
user_based_mae = 0.8   # Substitua pelo valor real após avaliação

labels = ['User-based', 'SVD']
rmse_scores = [user_based_rmse, svd_rmse]
mae_scores = [user_based_mae, svd_mae]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, rmse_scores, width, label='RMSE')
rects2 = ax.bar(x + width/2, mae_scores, width, label='MAE')

ax.set_ylabel('Score')
ax.set_title('Comparação de desempenho dos modelos')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()

NameError: name 'gs' is not defined